Entramos al apartado de limpieza y preparación de SQL

In [9]:
"""
!pip install scikit-learn
!pip install plotly
pip uninstall scikit-learn
pip uninstall plotly
"""

'\n!pip install scikit-learn\n!pip install plotly\npip uninstall scikit-learn\npip uninstall plotly\n'

In [10]:
import sys
print(sys.executable)

/home/kader/Descargas/Proyecto-empleo-main/.venv/bin/python


In [11]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install plotly

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install nbformat

Note: you may need to restart the kernel to use updated packages.


In [38]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import pickle
import requests
import sqlite3
import re
from sklearn.cluster import KMeans
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import nbformat
import webbrowser
import os


# Leer el archivo CSV
df = pd.read_csv('datos_pre_analisis.csv')

# Mostrar las primeras filas del DataFrame
print(df.head())

# Eliminar filas duplicadas
df = df.drop_duplicates()

# Función para limpiar y extraer rangos de salario
def limpieza_salario(rango_salarios):
    if isinstance(rango_salarios, str):
        rango_salarios = rango_salarios.replace('EUR', '').replace(',', '').strip()
        if ' - ' in rango_salarios:
            rango = rango_salarios.split(' - ')
            salario_min = int(rango[0].strip())
            salario_max = int(rango[1].strip())
        else:
            salario_min = salario_max = int(rango_salarios)
        return salario_min, salario_max
    else:
        return None, None

# Aplicar la función para extraer sueldo mínimo y máximo
df['sueldo_min'], df['sueldo_max'] = zip(*df['sueldo_min'].map(limpieza_salario))

# Limpieza de texto en la columna 'Título'
df['Título'] = df['Título'].str.replace(r'\(.*?\)', '', regex=True)
df['Título'] = df['Título'].str.replace(r'[|/]', '', regex=True)
df['Título'] = df['Título'].str.replace(r'\[.*?\]', '', regex=True)
df['Título'] = df['Título'].str.replace(r'[|/-]', '', regex=True)
df['Título'] = df['Título'].str.strip()

# Actualizar títulos de puesto
def actualizar_titulo(titulo):
    titulo = titulo.replace(' jr ', ' Junior ').replace(' sr ', ' Senior ')
    titulo = titulo.replace('jr ', 'Junior ').replace('sr ', 'Senior ')
    titulo = titulo.replace(' jr', ' Junior').replace(' sr', ' Senior')
    return titulo

df['Título'] = df['Título'].apply(actualizar_titulo)

# Definir expresiones regulares para empresas y sectores
regex_empresas = re.compile(r'(Google|Amazon|Facebook|Apple|Microsoft|IBM|Oracle|SAP|Salesforce|Meta)', re.IGNORECASE)
regex_sectores = re.compile(r'(IT|Finance|Healthcare|Retail|Consulting|Education|Manufacturing|Telecommunications|Media)', re.IGNORECASE)

# Funciones para extraer empresa y sector
def extraer_empresa(titulo):
    match = regex_empresas.search(titulo)
    return match.group(0) if match else None

def extraer_sector(titulo):
    match = regex_sectores.search(titulo)
    return match.group(0) if match else None

def limpiar_titulo(titulo):
    titulo = regex_empresas.sub('', titulo)
    titulo = regex_sectores.sub('', titulo)
    return ' '.join(titulo.split())

# Aplicar las funciones de limpieza
df['Empresa'] = df['Título'].apply(extraer_empresa)
df['Sector'] = df['Título'].apply(extraer_sector)
df['Título'] = df['Título'].apply(limpiar_titulo)

# Detectar idiomas en los títulos
def detectar_idiomas(titulo):
    if 'inglés' in titulo.lower() or 'english' in titulo.lower():
        return 'Inglés'
    elif 'francés' in titulo.lower() or 'french' in titulo.lower():
        return 'Francés'
    return None

df['Otro Idioma'] = df['Título'].apply(detectar_idiomas)

# Detectar modalidad de trabajo
df['Modalidad'] = df['Título'].apply(lambda x: 'Híbrido' if 'Híbrido' in x else ('Remoto' if 'Remoto' in x else ('Presencial' if 'Presencial' in x else None)))

# Diccionario de ubicaciones para normalización
ubicaciones_dict = {
    'Boadilla del Monte': 'Madrid',
    'Alcobendas': 'Madrid',
    'Leganés': 'Madrid',
    'Getafe': 'Madrid',
    'Barcelona, Centro': 'Cataluña',
    'Esplugues de Llobregat': 'Cataluña',
    'Barcelona': 'Cataluña',
    'Pozuelo de Alarcón': 'Madrid',
    'Tres Cantos': 'Madrid',
    'Villaviciosa de Odón': 'Madrid',
    'Sant Just Desvern': 'Cataluña',
    'Sant Cugat del Vallès':'Cataluña'
}

df['Ubicación'] = df['Ubicación'].replace(ubicaciones_dict)

# Clasificar roles según palabras clave
def clasificar_rol(titulo):
    titulo = titulo.lower()
    if 'data scientist' in titulo:
        return 'Data Scientist'
    elif 'data engineer' in titulo:
        return 'Data Engineer'
    elif 'manager data' in titulo or 'data manager' in titulo:
        return 'Data Manager'
    elif 'business intelligence' in titulo:
        return 'Business Intelligence'
    elif 'analyst' in titulo:
        return 'Data Analyst'
    elif 'consultant' in titulo:
        return 'Consultant'
    elif 'machine learning' in titulo:
        return 'Machine Learning Engineer'
    else:
        return 'Other'

df['Puesto'] = df['Título'].apply(clasificar_rol)

# Limpieza adicional del título
def limpiar_titulo_ciudades(titulo):
    ciudades = ["barcelona", "alicante", "madrid", "bilbao"]
    for ciudad in ciudades:
        titulo = re.sub(rf"\b{ciudad}\b", "", titulo, flags=re.IGNORECASE)
    titulo = re.sub(r"\b[a-zA-Z]{2,3}\d{3}\b", "", titulo)
    return re.sub(r"\s+", " ", titulo).strip()

df['Título'] = df['Título'].apply(limpiar_titulo_ciudades)

# DataFrame final procesado
df_new = df.copy()

# Mostrar el DataFrame limpio
print(df_new.head())




                                     Título                 Puesto  \
0                              data analyst           Data Analyst   
1             business intelligence analyst  Business Intelligence   
2                          research analyst           Data Analyst   
3  senior program manager analyst relations           Data Analyst   
4                              data analyst           Data Analyst   

   sueldo_min  sueldo_max Expertise Empresa Modalidad Sector Ubicación  \
0       50000       70000   Regular     NaN       NaN    NaN    Madrid   
1       50000       70000   Regular     NaN       NaN    NaN    Madrid   
2       30000       50000   Regular     NaN       NaN    NaN    Madrid   
3       50000       70000    Senior     NaN       NaN    NaN    Madrid   
4       50000       70000   Regular     NaN       NaN    NaN    Madrid   

                                         Descripción  \
0  Una empresa internacional y dinámica busca un ...   
1  An innovative edtec

In [15]:
df_new['Habilidades']

0                                            NaN
1             Análisis de datos, Data governance
2      Análisis de datos, Visualización de datos
3                                            NaN
4      Análisis de datos, Visualización de datos
                         ...                    
744                                          NaN
745                                          NaN
746                      Resolución de problemas
747                                          NaN
748                                          NaN
Name: Habilidades, Length: 749, dtype: object

In [16]:
# Diccionario de mapeo para estandarizar las respuestas de habilidades
skills_mapping = {
    r'\b(data analysis|análisis de datos|data science)\b': 'Análisis de datos',
    r'\b(machine learning|aprendizaje automático)\b': 'Machine Learning',
    r'\b(deep learning|aprendizaje profundo)\b': 'Deep Learning',
    r'\b(data visualization|visualización de datos)\b': 'Visualización de datos',
    r'\b(statistical analysis|análisis estadístico)\b': 'Análisis estadístico',
    r'\b(predictive modeling|modelado predictivo)\b': 'Modelado predictivo',
    r'\b(business intelligence|bi)\b': 'Business Intelligence',
    r'\b(reporting|reportes)\b': 'Reporting',
    r'\b(agile methodologies|metodologías ágiles)\b': 'Metodologías Ágiles',
    r'\b(optimización|optimization)\b': 'Optimización',
    r'\b(excel avanzado)\b': 'Excel Avanzado',
    r'\b(habilidades analíticas|analytical skills)\b': 'Habilidades analíticas',
    r'\b(problem solving|resolución de problemas)\b': 'Resolución de problemas',
    r'\b(neural networks|redes neuronales)\b': 'Redes Neuronales'
}

# Función para estandarizar la columna de habilidades
def standardize_skills(text):
    # Verificar si el texto es None o NaN y convertirlo a cadena vacía
    if not text or pd.isna(text):
        return ""
    
    # Asegurarse de que el texto sea una cadena de texto
    text = str(text)
    
    # Normalizar cada habilidad según el mapeo
    for pattern, standard_skill in skills_mapping.items():
        text = re.sub(pattern, standard_skill, text, flags=re.IGNORECASE)
    
    # Convertir en lista, eliminar duplicados y ordenar
    skills_list = list(set(text.split(',')))
    skills_list = sorted([skill.strip().capitalize() for skill in skills_list if skill.strip()])
    
    # Unir la lista en una sola cadena de texto
    return ', '.join(skills_list)

# Aplicar la función de estandarización a la columna 'Habilidades'
df_new['Habilidades'] = df_new['Habilidades'].apply(standardize_skills)

In [17]:
titulos_agrupados = df_new['Habilidades'].value_counts()

In [18]:
titulos_agrupados

Habilidades
                                                                            488
Análisis de datos                                                            45
Machine learning                                                             20
Análisis de datos, Visualización de datos                                    13
Análisis de datos, Machine learning                                          12
                                                                           ... 
Análisis de datos, Análisis de datos, Reporting, Resolución de problemas      1
Análisis de datos, Análisis estadístico, Machine learning                     1
Análisis de datos, Business intelligence, Machine learning                    1
Business intelligence, Reporting, Visualización de datos                      1
Deep learning, Deep learning                                                  1
Name: count, Length: 74, dtype: int64

In [19]:
# Diccionario de mapeo para estandarizar las respuestas de educación
education_mapping = {
    r'\b(grado)\b': 'Grado',
    r'\b(licenciatura)\b': 'Licenciatura',
    r'\b(engineering degree|ingeniería)\b': 'Ingeniería',
    r'\b(computer science degree)\b': 'Ciencias de la computación',
    r'\b(mathematics degree)\b': 'Matemáticas',
    r'\b(statistics degree)\b': 'Estadística',
    r'\b(ciencias económicas|economics)\b': 'Ciencias económicas'
}

# Función para estandarizar la columna de educación
def standardize_education(text):
    # Verificar si el texto es None o NaN antes de proceder
    if not text or pd.isna(text):
        return ""
    
    # Asegurarse de que el texto sea una cadena de texto
    text = str(text)
    
    # Normalizar cada grado según el mapeo
    for pattern, standard_education in education_mapping.items():
        text = re.sub(pattern, standard_education, text, flags=re.IGNORECASE)
    
    # Convertir en lista, eliminar duplicados, convertir a minúsculas y ordenar
    education_list = list(set(text.lower().split(',')))  # Convertir a minúsculas para evitar duplicados
    education_list = sorted([edu.strip().capitalize() for edu in education_list if edu.strip()])
    
    # Unir la lista en una sola cadena de texto
    return ', '.join(education_list)

# Aplicar la estandarización directamente a la columna "Educación"
df_new['Educación'] = df_new['Educación'].apply(standardize_education)

# Mostrar el DataFrame con la columna estandarizada
print(df_new['Educación'])

0      Carrera
1      Carrera
2             
3      Carrera
4      Carrera
        ...   
744           
745           
746    Carrera
747           
748           
Name: Educación, Length: 749, dtype: object


In [20]:
# Diccionario de mapeo para estandarizar las respuestas de herramientas
tools_mapping = {
    r'\b(python)\b': 'Python',
    r'\b(excel)\b': 'Excel',
    r'\b(tableau)\b': 'Tableau',
    r'\b(azure)\b': 'Azure',
    r'\b(docker)\b': 'Docker',
    r'\b(snowflake)\b': 'Snowflake',
    r'\b(pyspark)\b': 'PySpark',
    r'\b(scikit-learn|scikit learn)\b': 'Scikit-learn',
    r'\b(databricks)\b': 'Databricks',
    r'\b(google analytics)\b': 'Google Analytics',
    r'\b(adobe analytics)\b': 'Adobe Analytics',
    r'\b(power bi)\b': 'Power BI',
    r'\b(github)\b': 'GitHub',
    r'\b(tensorflow)\b': 'TensorFlow',
    r'\b(keras)\b': 'Keras',
    r'\b(pytorch)\b': 'PyTorch',
    r'\b(jupyter notebooks)\b': 'Jupyter Notebooks',
    r'\b(navision)\b': 'Navision',
    r'\b(refinitiv eikon|eikon)\b': 'Refinitiv Eikon',
    r'\b(bloomberg)\b': 'Bloomberg',
    r'\b(streamlit)\b': 'Streamlit'
}

# Función para estandarizar la columna de herramientas
def standardize_tools(text):
    # Verificar si el texto es None o NaN antes de proceder
    if not text or pd.isna(text):
        return ""
    
    # Asegurarse de que el texto sea una cadena
    text = str(text)
    
    # Normalizar cada herramienta según el mapeo
    for pattern, standard_tool in tools_mapping.items():
        text = re.sub(pattern, standard_tool, text, flags=re.IGNORECASE)
    
    # Convertir en lista, eliminar duplicados, convertir a minúsculas y ordenar
    tools_list = list(set(text.lower().split(',')))  # Convertir a minúsculas para evitar duplicados
    tools_list = sorted([tool.strip().capitalize() for tool in tools_list if tool.strip()])
    
    # Unir la lista en una sola cadena de texto
    return ', '.join(tools_list)

# Aplicar la estandarización directamente a la columna "Herramientas"
df_new['Herramientas'] = df_new['Herramientas'].apply(standardize_tools)

# Mostrar el DataFrame con la columna estandarizada
print(df_new['Herramientas'])

0      Excel, Python, R
1               Tableau
2        Excel, Tableau
3                      
4      Excel, Python, R
             ...       
744                    
745                    
746                    
747                    
748                    
Name: Herramientas, Length: 749, dtype: object


In [21]:
df_new['Herramientas']

0      Excel, Python, R
1               Tableau
2        Excel, Tableau
3                      
4      Excel, Python, R
             ...       
744                    
745                    
746                    
747                    
748                    
Name: Herramientas, Length: 749, dtype: object

In [22]:
df.columns

Index(['Título', 'Puesto', 'sueldo_min', 'sueldo_max', 'Expertise', 'Empresa',
       'Modalidad', 'Sector', 'Ubicación', 'Descripción', 'Servicios',
       'Otro Idioma', 'EntornoTEC', 'Beneficios', 'Habilidades',
       'Herramientas', 'Educación', 'sueldo_medio'],
      dtype='object')

In [23]:
df_new

,Título,Puesto,sueldo_min,sueldo_max,Expertise,Empresa,Modalidad,Sector,Ubicación,Descripción,Servicios,Otro Idioma,EntornoTEC,Beneficios,Habilidades,Herramientas,Educación,sueldo_medio
0,data analyst,Data Analyst,None,None,Regular,None,None,None,Madrid,Una empresa internacional y dinámica busca un ...,['Tarjeta de restaurante Seguro de salud Segur...,None,NaN,NaN,,"Excel, Python, R",Carrera,60000.0
1,business intelligence analyst,Business Intelligence,None,None,Regular,None,None,None,Madrid,An innovative edtech company is seeking a Busi...,['Flexible hybrid work environment Comprehensi...,None,NaN,NaN,"Análisis de datos, Data governance",Tableau,Carrera,60000.0
2,research analyst,Data Analyst,None,None,Regular,None,None,None,Madrid,An established industry player is seeking a Re...,['Competitive benefits Career development oppo...,None,NaN,NaN,"Análisis de datos, Visualización de datos","Excel, Tableau",,40000.0
3,senior program manager analyst relations,Data Analyst,None,None,Senior,None,None,None,Madrid,Join a forward-thinking company at the forefro...,['Equity (Restricted Stock Units)Life Insuranc...,None,NaN,NaN,,,Carrera,60000.0
4,data analyst,Data Analyst,None,None,Regular,None,None,None,Madrid,Join a forward-thinking company as a Data Anal...,['Flexible working hours Health insurance Life...,None,NaN,NaN,"Análisis de datos, Visualización de datos","Excel, Python, R",Carrera,60000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
744,data engineering manager,Data Engineer,None,None,Senior,None,None,None,Cataluña,Sin descripción,[],None,NaN,NaN,,,,40000.0
745,senior machine learning engineer,Machine Learning Engineer,None,None,Senior,None,None,None,Cataluña,Sin descripción,[],None,NaN,NaN,,,,60000.0
746,data engineering manager,Data Engineer,None,None,Senior,None,None,None,Cataluña,Join a forward-thinking company as a Data Engi...,['Lunch vouchers Health insurance Relocation p...,None,NaN,NaN,Resolución de problemas,,Carrera,40000.0
747,stibo step inhouse product data developer,Other,None,None,Regular,None,None,it,Cataluña,Join a forward-thinking company as an IT Stibo...,[' Stibo Step S Q L X M L X S D J S O N Data M...,None,NaN,NaN,,,,60000.0


In [24]:
df_final = pd.concat([df, df_new], axis=1)

df_final = df_final.dropna(subset=['Título'])

In [25]:
df_final['Herramientas']

,Herramientas,Herramientas
0,"Excel, Python, R","Excel, Python, R"
1,Tableau,Tableau
2,"Excel, Tableau","Excel, Tableau"
3,NaN,
4,"Excel, Python, R","Excel, Python, R"
...,...,...
744,NaN,
745,NaN,
746,NaN,
747,NaN,


In [26]:
df_final = df_final.drop_duplicates()

In [27]:
df_final.columns

Index(['Título', 'Puesto', 'sueldo_min', 'sueldo_max', 'Expertise', 'Empresa',
       'Modalidad', 'Sector', 'Ubicación', 'Descripción', 'Servicios',
       'Otro Idioma', 'EntornoTEC', 'Beneficios', 'Habilidades',
       'Herramientas', 'Educación', 'sueldo_medio', 'Título', 'Puesto',
       'sueldo_min', 'sueldo_max', 'Expertise', 'Empresa', 'Modalidad',
       'Sector', 'Ubicación', 'Descripción', 'Servicios', 'Otro Idioma',
       'EntornoTEC', 'Beneficios', 'Habilidades', 'Herramientas', 'Educación',
       'sueldo_medio'],
      dtype='object')

In [28]:

# Cargar los datos desde un archivo CSV
data = pd.read_csv('Datos_definitivos.csv')

# Verificar las primeras filas de los datos
print(data.head())

# Preparar los datos para el clustering (seleccionar las características relevantes)
X = data[['Puesto', 'Expertise', 'Sector']]

# Convertir variables categóricas a numéricas
X['Puesto'] = X['Puesto'].astype('category').cat.codes
X['Expertise'] = X['Expertise'].astype('category').cat.codes
X['Sector'] = X['Sector'].astype('category').cat.codes

# Aplicar K-means para segmentar los datos
kmeans = KMeans(n_clusters=3, random_state=0)
data['Segmento'] = kmeans.fit_predict(X)

# Visualizar la cantidad de ofertas por segmento
segment_counts = data['Segmento'].value_counts().reset_index()
segment_counts.columns = ['Segmento', 'Cantidad']

# Gráfico de barras mostrando la cantidad de ofertas por segmento
fig_barras = go.Figure(data=[
    go.Bar(x=segment_counts['Segmento'], y=segment_counts['Cantidad'], 
           marker=dict(color='royalblue'))
])

# Configurar el layout del gráfico de barras
fig_barras.update_layout(
    title='Cantidad de Ofertas por Segmento',
    xaxis_title='Segmento',
    yaxis_title='Cantidad de Ofertas'
)

# Guardar el gráfico como archivo HTML y abrirlo en el navegador
pio.write_html(fig_barras, 'grafico_barras.html')
webbrowser.open('grafico_barras.html')

# Gráfico de dispersión mostrando los diferentes segmentos
fig_dispersion = go.Figure()

# Agregar trazas para cada segmento
for segmento in data['Segmento'].unique():
    seg_data = data[data['Segmento'] == segmento]
    fig_dispersion.add_trace(go.Scatter(
        x=seg_data['Puesto'], 
        y=seg_data['Expertise'], 
        mode='markers',
        name=f'Segmento {segmento}',
        text=seg_data['Descripción'],  # Mostrar descripción al pasar el ratón
        marker=dict(size=10)
    ))

# Configurar el layout del gráfico de dispersión
fig_dispersion.update_layout(
    title='Resultados del Análisis de Segmentación',
    xaxis_title='Puesto',
    yaxis_title='Expertise',
    legend_title='Segmentos',
    hovermode='closest'
)

# Guardar el gráfico de dispersión como archivo HTML y abrirlo en el navegador
pio.write_html(fig_dispersion, 'grafico_dispersion.html')
webbrowser.open('grafico_dispersion.html')

# Verificar el número de columnas del DataFrame
print(f"El DataFrame tiene {data.shape[1]} columnas.")

# Asegurarte de que el DataFrame tiene el número correcto de columnas
# Si el DataFrame tiene más columnas de las que tienes en la lista, deberías recortar o completar
expected_columns = [
    "Título", "Puesto", 'sueldo_min', 'sueldo_max', "Expertise", "Empresa", "Modalidad", "Sector", 
    "Ubicación", "Descripción", "Servicios", "Otro_Idioma", "EntornoTEC", 
    "Beneficios", "Habilidades", "Herramientas", "Educación"
]

# Si el DataFrame tiene más columnas, agregar nombres genéricos para las columnas extra
if data.shape[1] > len(expected_columns):
    additional_columns = [f"Columna_extra_{i}" for i in range(len(expected_columns), data.shape[1])]
    data.columns = expected_columns + additional_columns
else:
    # Si el DataFrame tiene menos columnas, se recortan las columnas extras
    data.columns = expected_columns[:data.shape[1]]

# Mostrar los primeros registros del DataFrame con las columnas renombradas
print(data.head())


                                     Título                 Puesto  \
0                              data analyst           Data Analyst   
1             business intelligence analyst  Business Intelligence   
2                          research analyst           Data Analyst   
3  senior program manager analyst relations           Data Analyst   
4                              data analyst           Data Analyst   

   sueldo_min  sueldo_max Expertise  Empresa  Modalidad Sector Ubicación  \
0         NaN         NaN   Regular      NaN        NaN    NaN    Madrid   
1         NaN         NaN   Regular      NaN        NaN    NaN    Madrid   
2         NaN         NaN   Regular      NaN        NaN    NaN    Madrid   
3         NaN         NaN    Senior      NaN        NaN    NaN    Madrid   
4         NaN         NaN   Regular      NaN        NaN    NaN    Madrid   

                                         Descripción  ... Ubicación.1  \
0  Una empresa internacional y dinámica busca un 

/tmp/ipykernel_6586/3736954393.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Puesto'] = X['Puesto'].astype('category').cat.codes
/tmp/ipykernel_6586/3736954393.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['Expertise'] = X['Expertise'].astype('category').cat.codes
/tmp/ipykernel_6586/3736954393.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

El DataFrame tiene 37 columnas.
                                     Título                 Puesto  \
0                              data analyst           Data Analyst   
1             business intelligence analyst  Business Intelligence   
2                          research analyst           Data Analyst   
3  senior program manager analyst relations           Data Analyst   
4                              data analyst           Data Analyst   

   sueldo_min  sueldo_max Expertise  Empresa  Modalidad Sector Ubicación  \
0         NaN         NaN   Regular      NaN        NaN    NaN    Madrid   
1         NaN         NaN   Regular      NaN        NaN    NaN    Madrid   
2         NaN         NaN   Regular      NaN        NaN    NaN    Madrid   
3         NaN         NaN    Senior      NaN        NaN    NaN    Madrid   
4         NaN         NaN   Regular      NaN        NaN    NaN    Madrid   

                                         Descripción  ...  \
0  Una empresa internacional 

In [29]:
df_final.columns

Index(['Título', 'Puesto', 'sueldo_min', 'sueldo_max', 'Expertise', 'Empresa',
       'Modalidad', 'Sector', 'Ubicación', 'Descripción', 'Servicios',
       'Otro Idioma', 'EntornoTEC', 'Beneficios', 'Habilidades',
       'Herramientas', 'Educación', 'sueldo_medio', 'Título', 'Puesto',
       'sueldo_min', 'sueldo_max', 'Expertise', 'Empresa', 'Modalidad',
       'Sector', 'Ubicación', 'Descripción', 'Servicios', 'Otro Idioma',
       'EntornoTEC', 'Beneficios', 'Habilidades', 'Herramientas', 'Educación',
       'sueldo_medio'],
      dtype='object')

In [39]:

# Función para aplicar encoders a las columnas categóricas del dataset específico
def apply_encoders_to_custom_data(df):
    # Identificar las columnas numéricas y categóricas
    categorical_columns = df.select_dtypes(include=['object']).columns
    numerical_columns = df.select_dtypes(exclude=['object']).columns

    # Crear un diccionario para almacenar los encoders
    label_encoders = {}
    onehot_encoders = {}

    # Aplicar LabelEncoder a las columnas categóricas con pocas categorías
    for col in categorical_columns:
        if df[col].nunique() <= 10:  # Umbral de cardinalidad baja
            label_encoder = LabelEncoder()
            df[col] = label_encoder.fit_transform(df[col])
            label_encoders[col] = label_encoder

    # Aplicar OneHotEncoder a las columnas con alta cardinalidad
    for col in categorical_columns:
        if df[col].nunique() > 10:  # Umbral de cardinalidad alta
            onehot_encoder = OneHotEncoder(sparse_output=False)  # Usar sparse_output en lugar de sparse
            transformed = onehot_encoder.fit_transform(df[[col]])
            # Crear nuevas columnas con los valores transformados
            df = pd.concat([df, pd.DataFrame(transformed, columns=[f"{col}_{i}" for i in range(transformed.shape[1])])], axis=1)
            df = df.drop(columns=[col])  # Eliminar la columna original
            onehot_encoders[col] = onehot_encoder

    # Guardar los LabelEncoders en archivos pickle
    for col, encoder in label_encoders.items():
        with open(f"{col}_label_encoder.pkl", 'wb') as f:
            pickle.dump(encoder, f)

    # Guardar los OneHotEncoders en archivos pickle
    for col, encoder in onehot_encoders.items():
        with open(f"{col}_onehot_encoder.pkl", 'wb') as f:
            pickle.dump(encoder, f)

    return df, label_encoders, onehot_encoders


# Comprobar si el archivo existe antes de intentar cargarlo
file_path = 'datos_pre_analisis.csv'

if os.path.exists(file_path):
    # Cargar el dataset
    df = pd.read_csv(file_path)

    # Aplicar encoders al dataset
    df_encoded, label_encoders, onehot_encoders = apply_encoders_to_custom_data(df)

    # Mostrar el dataframe codificado
    print(df_encoded.head())
else:
    print(f"El archivo {file_path} no se encuentra en la ubicación especificada.")


   sueldo_min  sueldo_max  Expertise  Empresa  Modalidad  Ubicación  \
0       50000       70000          3        6          3          2   
1       50000       70000          3        6          3          2   
2       30000       50000          3        6          3          2   
3       50000       70000          4        6          3          2   
4       50000       70000          3        6          3          2   

   Otro Idioma  Educación  sueldo_medio  Título_0  ...  Herramientas_89  \
0            2          0       60000.0       0.0  ...              0.0   
1            2          0       60000.0       0.0  ...              0.0   
2            2          6       40000.0       0.0  ...              0.0   
3            2          0       60000.0       0.0  ...              0.0   
4            2          0       60000.0       0.0  ...              0.0   

   Herramientas_90  Herramientas_91  Herramientas_92  Herramientas_93  \
0              0.0              0.0              

In [40]:

# Conectar a la base de datos SQLite
conn = sqlite3.connect('database.db')
cursor = conn.cursor()

# Eliminamos y creamos la tabla
cursor.execute("DROP TABLE IF EXISTS bbdd_ofertas")
cursor.execute('''
    CREATE TABLE bbdd_ofertas (
        Título TEXT,
        Puesto TEXT,
        Expertise TEXT,
        Empresa TEXT,
        Modalidad TEXT,
        Sueldo_medio TEXT,
        Sector TEXT,
        Ubicación TEXT,
        Descripción TEXT,
        Servicios TEXT,
        Otro_Idioma TEXT,
        EntornoTEC TEXT,
        Beneficios TEXT,
        Habilidades TEXT,
        Herramientas TEXT,
        Educación TEXT,
        cluster_label INTEGER
    )
''')

# Asegúrate de que el DataFrame `df_final` esté limpio y preparado para clustering
if 'df_final' in globals():
    if 'Sueldo_medio' in df_new.columns:
        # Convertir 'Sueldo_medio' a numérico, forzando los valores no convertibles a NaN y luego reemplazándolos por 0
        df_new['Sueldo_medio'] = pd.to_numeric(df_new['Sueldo_medio'], errors='coerce').fillna(0)

        # Convertir variables categóricas a numéricas usando One-Hot Encoding
        df_clustering = pd.get_dummies(df_new[['Sueldo_medio', 'Ubicación', 'Expertise', 'Modalidad']])
        
        # Normalizar los datos
        scaler = StandardScaler()
        df_scaled = scaler.fit_transform(df_clustering)
        
        # Aplicar K-Means clustering
        kmeans = KMeans(n_clusters=5, random_state=0)
        df_new['cluster_label'] = kmeans.fit_predict(df_scaled)
        
        # Detectar duplicados y evitar insertar datos repetidos
        query_existentes = "SELECT Título, Empresa, Ubicación FROM bbdd_ofertas"
        df_existentes = pd.read_sql(query_existentes, conn)
        
        # Filtrar los datos nuevos que no están en la base de datos
        df_final_limpio = df_new.merge(df_existentes, on=['Título', 'Empresa', 'Ubicación'], 
                                         how='left', indicator=True)
        df_final_limpio = df_final_limpio[df_final_limpio['_merge'] == 'left_only'].drop(columns='_merge')
        
        if not df_final_limpio.empty:
            # Insertar el DataFrame limpio en la tabla SQLite
            df_final_limpio.to_sql('bbdd_ofertas', conn, if_exists='append', index=False)
            print(f"Se insertaron {len(df_final_limpio)} nuevos registros.")
        else:
            print("No se encontraron nuevos registros para insertar.")

        # Verificar que los datos se hayan guardado correctamente
        df_check = pd.read_sql("SELECT Título, Sueldo_medio, cluster_label FROM bbdd_ofertas LIMIT 5", conn)
        print("Datos insertados (muestra):")
        print(df_check)
        
    else:
        print("La columna 'Sueldo_medio' no se encuentra en df_final.")
else:
    print("El DataFrame 'df_final' no está definido o cargado correctamente.")

# Cerrar la conexión temporal
conn.close()

# Consultas de Verificación
# Conectar a la base de datos SQLite
conn = sqlite3.connect('database.db')

# Leer los datos de la tabla 'bbdd_ofertas' en un DataFrame de pandas
query = "SELECT * FROM bbdd_ofertas LIMIT 5"
df_check = pd.read_sql(query, conn)
print("Datos de muestra:")
print(df_check)

# Consultar los datos donde la ubicación es "Madrid"
query_madrid = "SELECT * FROM bbdd_ofertas WHERE Ubicación = 'Madrid'"
df_madrid = pd.read_sql(query_madrid, conn)
print("Registros de Madrid:")
print(df_madrid)

# Cerrar la conexión
conn.close()


La columna 'Sueldo_medio' no se encuentra en df_final.
Datos de muestra:
Empty DataFrame
Columns: [Título, Puesto, Expertise, Empresa, Modalidad, Sueldo_medio, Sector, Ubicación, Descripción, Servicios, Otro_Idioma, EntornoTEC, Beneficios, Habilidades, Herramientas, Educación, cluster_label]
Index: []
Registros de Madrid:
Empty DataFrame
Columns: [Título, Puesto, Expertise, Empresa, Modalidad, Sueldo_medio, Sector, Ubicación, Descripción, Servicios, Otro_Idioma, EntornoTEC, Beneficios, Habilidades, Herramientas, Educación, cluster_label]
Index: []


In [ ]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 748 entries, 0 to 748
Data columns (total 36 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Título        748 non-null    object 
 1   Puesto        748 non-null    object 
 2   sueldo_min    0 non-null      object 
 3   sueldo_max    0 non-null      object 
 4   Expertise     748 non-null    object 
 5   Empresa       0 non-null      object 
 6   Modalidad     0 non-null      object 
 7   Sector        1 non-null      object 
 8   Ubicación     748 non-null    object 
 9   Descripción   748 non-null    object 
 10  Servicios     748 non-null    object 
 11  Otro Idioma   14 non-null     object 
 12  EntornoTEC    59 non-null     object 
 13  Beneficios    46 non-null     object 
 14  Habilidades   261 non-null    object 
 15  Herramientas  465 non-null    object 
 16  Educación     282 non-null    object 
 17  sueldo_medio  748 non-null    float64
 18  Título        748 non-null    objec